In [1]:
from sentence_transformers import SentenceTransformer
import os
import re
import codecs
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import datetime
from scipy.spatial.distance import cosine

%matplotlib inline

# !pip install transformers
# !pip install sentence_transformers

In [5]:
name_of_folder_for_processing = 'sampled_raw'
Country_to_process = 'IN'

In [6]:
base_dir = os.path.join(os.getcwd(), 'data', name_of_folder_for_processing) 
countries_list = os.listdir(base_dir) # dir is your directory path
summary = []

for con in countries_list:
    if(os.path.isdir(os.path.join(base_dir,con))):
        print(con, end=", ", flush=True)
        publishers = os.listdir(os.path.join(base_dir,con))
        for pub in publishers:
            subdir = os.path.join(base_dir, con, pub)
            length = sum([len(files) for r, d, files in os.walk(subdir)])
            summary.append([con, pub, length])

summary_df = pd.DataFrame(summary, columns = ['country','publisher','count'])     

PK, US, HK, GH, NG, KE, NZ, BD, IN, LK, CA, ZA, SG, PH, GB, MY, AU, IE, JM, TZ, 

In [7]:
summary_df

,country,publisher,count
0,PK,oyeyeah-com,711
1,PK,nation-com-pk,806
2,PK,business-recorder-press-release-registration-blog,1390
3,PK,paktribune-com,337
4,PK,aaj-tv-press-release-blog,143
...,...,...,...
2863,TZ,goal-com,1315
2864,TZ,rapid-tv-news,1632
2865,TZ,ippmedia-com,1115
2866,TZ,daily-news-the-national-newspaper,707


In [ ]:
# idx = summary_df.groupby(['country'])['count'].transform(max) == summary_df['count']
# max_table = summary_df[idx]
# max_table

In [8]:
overall_dict = {}

for index, row in summary_df.iterrows():
    country, publisher = row[0], row[1]
    if country != Country_to_process:
        pass
    else:
        
        print(country, end=", ", flush=True)
        directory = os.path.join(base_dir, country, publisher)
        df_dict = {"id": [], "title": [], "publisher": [], "article_text":[],"url":[], "path":[]}
        if not os.path.isdir(directory):
            pass
        else:
            for entry in os.scandir(directory):
                if(os.path.isdir(entry)):
                    for entry_2 in os.scandir(entry.path):
                        try:
                            with open(entry_2, "r") as f:
                                article_id = f.readline().strip()
                                article_title = f.readline().strip()
                                publisher = f.readline().strip()
                                url = f.readline()
                                f.readline()
                                article_text = f.readline().strip()
                                df_dict['url'].append(url)
                                df_dict['id'].append(article_id)
                                df_dict['title'].append(article_title)
                                df_dict['publisher'].append(publisher)
                                df_dict['article_text'].append(article_text)
                                df_dict['path'].append(entry_2)
                        except:
                            pass
            sample_df = pd.DataFrame.from_dict(df_dict)
            #sample_df = sample_df.sample(n=5, random_state=1)

            if country not in overall_dict.keys():
                overall_dict[country] = {}
            overall_dict[country][publisher] = sample_df


IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, 

## Preprocessing


In [9]:
# import a clean function data
def clean_text(text):
    return text.strip()

def text_initial_preproecss(df):

    df = df.copy()
    df = df[~pd.to_numeric(df['id'], errors='coerce').isnull()]
    df["id"] = df["id"].astype(int)
    df['article_text'] = df.article_text.str.lower()
    df["article_text"] = df["article_text"].apply(clean_text)
    df['text'] = df['article_text']
    reports = df
    
    # Puncutation preprocesing
    reports['text'] = reports.text.str.replace('{', '')
    reports['text'] = reports.text.str.replace('}', '')
    reports['text'] = reports.text.str.replace("\n", '')
    reports['text'] = reports.text.str.rstrip("\n") #remove empty lines
    reports['text'] = reports.text.str.replace("@ @ @ @ @ @ @ @ @ @ ", '')
    reports['text'] = reports.text.str.replace(" @", '')
    reports['text'] = reports.text.str.replace(" '", "'")
    reports['text'] = reports.text.str.replace("\"", "")
    reports['text'] = reports.text.str.replace(",", "")
    reports['text'] = reports.text.str.replace("(", "")
    reports['text'] = reports.text.str.replace(")", "")
    reports['text'] = reports.text.str.replace(" <p>", ".")
    reports['text'] = reports.text.str.replace(" <h>", ".")
    reports['text'] = reports.text.str.replace("<p>", "")
    reports['text'] = reports.text.str.replace("<h>", "")
    reports['text'] = reports.text.str.replace('<', '')
    reports['text'] = reports.text.str.replace('>', '')
    reports['text'] = reports.text.str.replace(":", "")
    reports['text'] = reports.text.str.replace("?", ".")
    reports['text'] = reports.text.str.replace("!", ".")
    reports['text'] = reports.text.str.replace(r"\.\s[\.\s]+", ". ") #converting . . to .
    reports['text'] = reports.text.str.replace(r"\.+", ".") #converting ... to .
    reports['text'] = reports.text.str.replace("--", "") 
    reports['text'] = reports.text.str.replace("-", " ")
    reports['text'] = reports.text.str.replace(" +", " ")
    reports['text'] = reports.text.str.replace(" n't", "n't")
    
    return reports.copy()

In [10]:
import platform
os_name = platform.system()

def save_file(report,base):
#     print(report)
#     print(type(report))
    report = report.squeeze()
        
    full_path = report.publisher
    file_name = str(report.id) + ".txt"
    
    if os_name == 'Windows':
        
        base_dir = 'data\preprocessed\\{}'.format(base)
        con = report.path.path.split('{0}\\'.format(name_of_folder_for_processing))[1]
        
    else:
        
        base_dir = 'data/preprocessed/{}'.format(base)
        con = report.path.path.split('/{0}/'.format(name_of_folder_for_processing))[1]
    
    path = os.path.join(base_dir,con)
    
    full_path = os.path.dirname(path)
    file_name = os.path.basename(path)

    pathlib.Path(full_path).mkdir(parents=True, exist_ok=True)
    
    with codecs.open(f"{full_path}/{file_name}", "w", encoding = "utf-8") as f:
        id_number = str(report.id) if not pd.isna(report.id) else ""
        title = report.title if not pd.isna(report.title) else ""
        website = report.publisher if not pd.isna(report.publisher) else ""
        url = report.url if not pd.isna(report.url) else ""
        f.writelines([id_number, "\n",
                      title, "\n",
                      website, "\n",
                      url, "\n\n", report.text])
        f.close()

In [11]:
def ngram_preprocess(df):
    time_taken = []
    
#     reports_temp = df[df['publisher']==article][['id', 'text','text_vect']].copy()
    if len(df) > 500:
        reports_temp = df.sample(n=500, random_state=1).copy()
    else:
        reports_temp = df.copy()
        
    reports_temp['text_vect'] = reports_temp.text.apply(sent_tokenize)
#     display(reports_temp.head(3))
    word_vectorizer = CountVectorizer(ngram_range=(5,5), stop_words=[])
    flatten = [item for sublist in reports_temp['text_vect'] for item in sublist]
    sparse_matrix = word_vectorizer.fit_transform(flatten)

    frequency = sum(sparse_matrix).toarray()[0]
    frequency_df = pd.DataFrame(frequency, index=word_vectorizer.get_feature_names(),columns = ['frequency']).sort_values(by=['frequency'],ascending=False)

    freq_above_10 = frequency_df[frequency_df['frequency'] >= max((len(reports_temp)/10),3)] #len(MAXSIZE)/2
    phrase_list = list(freq_above_10.index)

    sentences_to_remove = []

    for phrase in phrase_list:
#         removing_sent = set([sent for sent in flatten if phrase in sent])
        removing_sent = set([sent for sent in flatten if ((phrase in sent) or (len(sent.split()) < 4))])

        for sent in removing_sent:
            sentences_to_remove.append(sent)

    if("." in sentences_to_remove):
        while("." in sentences_to_remove):
            sentences_to_remove.remove(".")
    if(" ." in sentences_to_remove):
        while(" ." in sentences_to_remove):
            sentences_to_remove.remove(" .")

#    print(sentences_to_remove)
    sentences_to_remove = list(set(sentences_to_remove))
    sentences_to_remove.sort(key= len, reverse = True)  # sort starting by largest sentence, in case smaller sentence get chosen beforehand

    for sent in sentences_to_remove:
        df.text = df.text.apply(lambda x: str(x).replace(sent, ""))

#     print(sentences_to_remove)
    return df



In [12]:
def cosine_preprocess(df, model, threshold):

    sent_removed = []
    time_taken = []
    new_txts = []
    
    for i in range(len(df)):
        
        idx_remove = []
        orig_txt = df['text'].iloc[i]
        sent_text = nltk.sent_tokenize(orig_txt) #Sentence Segmentation
        docu_embeddings = model.encode(orig_txt) #Document Embedding
        sentence_embeddings_ = model.encode(sent_text) #Sentence Embedding

        for k in range(len(sentence_embeddings_)):
            
            sim = cosine(docu_embeddings, sentence_embeddings_[k])

            if sim > threshold:
                idx_remove.append(k)

        idx_remove.sort()
        idx_remove.reverse()

        for j in idx_remove:
            sent_removed.append(sent_text[j])
            del sent_text[j]

        new_txt = ' '.join(sent_text)
        new_txts.append(new_txt)
        
    df['text'] = new_txts

    return df

## Saving original file, to do comparison later

In [13]:
for keys1 in overall_dict.keys():
    for keys2 in overall_dict[keys1].keys():
        df = overall_dict[keys1][keys2]
        df = text_initial_preproecss(df)

        df.apply(save_file,args=('orig',),axis=1)
        

KeyboardInterrupt: 

In [14]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
import string
from textblob import TextBlob, Word

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

stopwords.words('english')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def remove_stopwords(text):
    word_tokens = word_tokenize(text) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

def remove_digits(text):
    text = re.sub(r'\d+','', text)
    return text

def remove_punct(text):
    word_tokens = word_tokenize(text) 
    filtered_sentence = [w for w in word_tokens if not w in string.punctuation]
    return ' '.join(filtered_sentence)

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def nltk_lemmatizer(text):
    '''
    lemmatize each word in sentence if the word is not punctuation
    Below version is just for ease of understanding
    '''
    words = word_tokenize(text)
    lem_words = []
    for word in words:
        if word not in string.punctuation:
            lem_words.append(lemmatizer.lemmatize(word, get_wordnet_pos(word)))

    return ' '.join(lem_words)

def lemmatize_with_postag(text):
    sent = TextBlob(text)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hojinlee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hojinlee/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hojinlee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## N-Gram

In [15]:
cosine_process = False
N_gram_process = True
merged_process = True if cosine_process and N_gram_process else False
cosine_threshold=0.95

time_taken = []
text_list = []

if merged_process:
    base='merged'
elif cosine_process:
    base='cosine'
else:
    base='ngram'

print("Methodology: {}".format(base))
  
for keys1 in overall_dict.keys():
    if keys1 != Country_to_process:
        pass
    else:
        for keys2 in overall_dict[keys1].keys(): # i.e. 'US', "CA"

            if keys1 =='IN':
                a = datetime.datetime.now()

                df = overall_dict[keys1][keys2]
                df = text_initial_preproecss(df)

                b = datetime.datetime.now()

                if merged_process:
                    df = ngram_preprocess(df)
                    df = cosine_preprocess(df,model,cosine_threshold)

                elif cosine_process:
                    df = cosine_preprocess(df,model,cosine_threshold)

                elif N_gram_process:
                    df = ngram_preprocess(df)


                df.apply(save_file,args = (base,),axis=1)

                df.text = df.text.apply(remove_stopwords)
                df.text = df.text.apply(remove_punct)
                df.text = df.text.apply(remove_digits)
                df.text = df.text.apply(lemmatize_with_postag)

                #saving files
                df.apply(save_file,args = ('lemmatized',),axis=1)

                c = datetime.datetime.now()

                print("Running time, country: {}, publisher: {}, punct_t: {}, text_proc_t: {}, total_t: {}".format(keys1,keys2, b-a, c-b, c-a))
                time_taken.append(c-a)

print(np.mean(time_taken))
print(np.sum(time_taken))

Methodology: ngram
Running time, country: IN, publisher: News Nation, punct_t: 0:00:00.073108, text_proc_t: 0:00:11.222836, total_t: 0:00:11.295944
Running time, country: IN, publisher: Gulte, punct_t: 0:00:00.031223, text_proc_t: 0:00:02.036926, total_t: 0:00:02.068149


KeyboardInterrupt: 

## Cosine

In [ ]:
# cosine_process = True
# N_gram_process = False
# merged_process = True if cosine_process and N_gram_process else False
# cosine_threshold=0.95

# model = SentenceTransformer('sentence-transformers/bert-base-nli-stsb-mean-tokens')

# time_taken = []
# text_list = []

# if merged_process:
#     base='merged'
# elif cosine_process:
#     base='cosine'
# else:
#     base='ngram'

# print("Methodology: {}".format(base))
  
# for keys in overall_dict.keys(): # i.e. 'US', "CA"
    
#     a = datetime.datetime.now()
    
#     df = overall_dict[keys]   
#     df = text_initial_preproecss(df)
    
#     b = datetime.datetime.now()
    
# #     df.apply(save_file_initial,args = (base,),axis=1)

# #     text_list.append(df.iloc[1])
#     if merged_process:
#         df = ngram_preprocess(df)
#         df = cosine_preprocess(df,model,cosine_threshold)

#     elif cosine_process:
#         df = cosine_preprocess(df,model,cosine_threshold)

#     elif N_gram_process:
#         df = ngram_preprocess(df)

    
#     #saving files
#     df.apply(save_file,args = (base,),axis=1)

#     c = datetime.datetime.now()
    
#     print("Running time, country: {}, punct_t: {}, text_proc_t: {}, total_t: {}".format(keys, b-a, c-b, c-a))
#     time_taken.append(c-a)

# print(np.mean(time_taken))
# print(np.sum(time_taken))

## Merged

In [ ]:
# cosine_process = True
# N_gram_process = True
# merged_process = True if cosine_process and N_gram_process else False
# cosine_threshold=0.95

# model = SentenceTransformer('sentence-transformers/bert-base-nli-stsb-mean-tokens')

# time_taken = []
# text_list = []

# if merged_process:
#     base='merged'
# elif cosine_process:
#     base='cosine'
# else:
#     base='ngram'

# print("Methodology: {}".format(base))
  
# for keys in overall_dict.keys(): # i.e. 'US', "CA"
    
#     a = datetime.datetime.now()
    
#     df = overall_dict[keys]   
#     df = text_initial_preproecss(df)
    
#     b = datetime.datetime.now()
    
# #     df.apply(save_file_initial,args = (base,),axis=1)

# #     text_list.append(df.iloc[1])
#     if merged_process:
#         df = ngram_preprocess(df)
#         df = cosine_preprocess(df,model,cosine_threshold)

#     elif cosine_process:
#         df = cosine_preprocess(df,model,cosine_threshold)

#     elif N_gram_process:
#         df = ngram_preprocess(df)

    
#     #saving files
#     df.apply(save_file,args = (base,),axis=1)

#     c = datetime.datetime.now()
    
#     print("Running time, country: {}, punct_t: {}, text_proc_t: {}, total_t: {}".format(keys, b-a, c-b, c-a))
#     time_taken.append(c-a)

# print(np.mean(time_taken))
# print(np.sum(time_taken))